<a href="https://colab.research.google.com/github/Teasotea/BioNER-and-RD/blob/main/RelationExtractionBetweenEntities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [ ]:
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import statistics as stats

In [ ]:
import nltk
from nltk.tokenize import TreebankWordTokenizer as twt
import spacy
from spacy import displacy

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string 

In [ ]:
import pickle
from google.colab import files

In [ ]:
import bs4
import requests
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline


In [ ]:
!sudo apt install nvidia-smi

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package nvidia-smi is a virtual package provided by:
  nvidia-utils-515 515.48.07-0ubuntu1
  nvidia-utils-510 510.73.08-0ubuntu1
  nvidia-utils-470 470.129.06-0ubuntu1
  nvidia-utils-450 450.191.01-0ubuntu1
  nvidia-utils-495 495.29.05-0ubuntu1
  nvidia-utils-465 465.19.01-0ubuntu1
  nvidia-utils-460 460.106.00-0ubuntu1
  nvidia-utils-455 455.45.01-0ubuntu1
  nvidia-utils-440 440.118.02-0ubuntu1
  nvidia-utils-418 418.226.00-0ubuntu1
  nvidia-utils-410 410.129-0ubuntu1
  nvidia-utils-510-server 510.73.08-0ubuntu0.18.04.1
  nvidia-utils-470-server 470.129.06-0ubuntu0.18.04.1
  nvidia-utils-450-server 450.191.01-0ubuntu0.18.04.1
  nvidia-utils-418-server 418.226.00-0ubuntu0.18.04.2
  nvidia-utils-390 390.151-0ubuntu0.18.04.1
You should explicitly select one to install.

E: Package 'nvidia-smi' has no installation candidate


In [29]:
!nvidia-smi

Sat Jun 25 09:03:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    30W /  70W |   5934MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
import torch
print('pytorch version:', torch.__version__)
print('GPU:',torch.cuda.get_device_name(0))

pytorch version: 1.11.0+cu113
GPU: Tesla T4


# Load Data

In [ ]:
with open('/content/data0.txt') as f:
      eval_article = f.read()

In [ ]:
train_df = pd.read_csv('/content/cdr_dner_train_df.csv')
test_df = pd.read_csv('/content/cdr_dner_test_df.csv')
dev_df = pd.read_csv('/content/cdr_dner_dev_df.csv')

In [ ]:
CID_train_df = train_df[train_df['xloc'] == "CID"].reset_index(drop=True)
DNER_train_df = train_df[train_df['xloc'] != "CID"].reset_index(drop=True)

CID_test_df = test_df[test_df['xloc'] == "CID"].reset_index(drop=True)
DNER_test_df = test_df[test_df['xloc'] != "CID"].reset_index(drop=True)

CID_dev_df = dev_df[dev_df['xloc'] == "CID"].reset_index(drop=True)
DNER_dev_df = dev_df[dev_df['xloc'] != "CID"].reset_index(drop=True)

In [ ]:
iob_train = pd.read_csv('/content/iob_train.csv',converters={"tokens": lambda x: x.strip("[]").replace("'","").split(", "), "iob_tags": lambda x: x.strip("[]").replace("'","").split(", ")})
iob_test = pd.read_csv('/content/iob_test.csv',converters={"tokens": lambda x: x.strip("[]").replace("'","").split(", "), "iob_tags": lambda x: x.strip("[]").replace("'","").split(", ")})
iob_dev = pd.read_csv('/content/iob_dev.csv',converters={"tokens": lambda x: x.strip("[]").replace("'","").split(", "), "iob_tags": lambda x: x.strip("[]").replace("'","").split(", ")})

In [33]:
iob_dev

,tokens,iob_tags
0,"[Tricuspid, valve, regurgitation, and, lithium, carbonate, toxicity, in, a, newborn, infant., A, newborn, with, massive, tricuspid, regurgitation, ,, atrial, flutter, ,, congestive, heart, failure...","[B-Disease, I-Disease, I-Disease, O, B-Chemical, I-Chemical, B-Disease, O, O, O, O, O, O, O, O, B-Disease, I-Disease, O, B-Disease, I-Disease, O, B-Disease, I-Disease, I-Disease, O, O, O, O, O, B-..."
1,"[Phenobarbital-induced, dyskinesia, in, a, neurologically-impaired, child., A, 2-year-old, child, with, known, neurologic, impairment, developed, a, dyskinesia, soon, after, starting, phenobarbita...","[B-Chemical, B-Disease, O, O, B-Disease, O, O, O, O, O, O, B-Disease, I-Disease, O, O, B-Disease, O, O, O, B-Chemical, O, O, B-Disease, O, O, O, B-Disease, I-Disease, O, O, O, O, O, O, O, O, B-Che..."
2,"[Acute, changes, of, blood, ammonia, may, predict, short-term, adverse, effects, of, valproic, acid., Valproic, acid, (, VPA, ), was, given, to, 24, epileptic, patients, who, were, already, being,...","[O, O, O, O, B-Chemical, O, O, O, O, O, O, B-Chemical, O, B-Chemical, I-Chemical, O, B-Chemical, O, O, O, O, O, B-Disease, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Chemical, O, O, O, O, O, O..."
3,"[Effects, of, calcitonin, on, rat, extrapyramidal, motor, system, :, behavioral, and, biochemical, data., The, effects, of, i.v.c., injection, of, human, and, salmon, calcitonin, on, biochemical, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Chemical, B-Disease, O, O, O, O, O, B-Chemical, B-Di..."
4,"[Development, of, isoproterenol-induced, cardiac, hypertrophy., The, development, of, cardiac, hypertrophy, was, studied, in, adult, female, Wistar, rats, following, daily, subcutaneous, injection...","[O, O, B-Chemical, B-Disease, O, O, O, O, B-Disease, I-Disease, O, O, O, O, O, O, O, O, O, O, O, O, B-Chemical, O, B-Chemical, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
487,"[Reversible, cerebral, lesions, associated, with, tiazofurin, usage, :, MR, demonstration., Tiazofurin, is, an, experimental, chemotherapeutic, agent, currently, undergoing, clinical, evaluation.,...","[O, B-Disease, I-Disease, O, O, B-Chemical, O, O, O, O, B-Chemical, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Disease, I-Disease, O, O, O, O, O, O, O, O, O, O, O, O, O, O..."
488,"[Antagonism, of, diazepam-induced, sedative, effects, by, Ro15-1788, in, patients, after, surgery, under, lumbar, epidural, block., A, double-blind, placebo-controlled, investigation, of, efficacy...","[O, O, B-Chemical, O, O, O, B-Chemical, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Chemical, O, O, O, O, O, B-Chemical, O, O, O, O, O, O, O, O, O, O, O, O, ..."
489,"[Enhanced, stimulus-induced, neurotransmitter, overflow, in, epinephrine-induced, hypertensive, rats, is, not, mediated, by, prejunctional, beta-adrenoceptor, activation., The, present, study, exa...","[O, O, O, O, O, B-Chemical, B-Disease, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Chemical, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Chemical, O, O, B..."
490,"[Ocular, manifestations, of, juvenile, rheumatoid, arthritis., We, followed, 210, cases, of, juvenile, rheumatoid, arthritis, closely, for, eleven, years., Thirty-six, of, the, 210, patients, (, 1...","[O, O, O, B-Disease, I-Disease, O, O, O, O, O, O, B-Disease, I-Disease, I-Disease, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Disease, B-Disease, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O..."


# BERT-sklearn Model

### Pre-process and import libraries

In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

In [ ]:
# !pip install --upgrade botocore==1.23.26
# !pip install --upgrade urllib3==1.22.0

In [ ]:
# !pip install --upgrade sagemaker

In [ ]:
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
train = iob_train.append(iob_dev, ignore_index=True)
train = train.reset_index(drop = True)
print("Train and dev data: %d sentences, %d tokens"%(len(train), len(flatten(train.tokens.tolist()))))
print("Test data: %d sentences, %d tokens"%(len(iob_test), len(flatten(iob_test.tokens))))

In [ ]:
X_train, y_train = train.tokens, train.iob_tags
X_test, y_test = iob_test.tokens, iob_test.iob_tags

In [ ]:
y_train

In [ ]:
label_list = list(np.unique(flatten(y_train)))
print("\nNER tags:", label_list)

In [ ]:
train.head()

### Finetune SciBERT

1) Use `'scibert-scivocab-cased'` model. [GitHub](https://github.com/allenai/scibert), [paper](https://arxiv.org/pdf/1903.10676.pdf)

2) Set the **`ignore_label`** =`['O']`. Exclude the non named entities.

3) Set the **` max_seq_len`** - high enough to cover lengths of the token lists, not too high because of GPU limitations

In [ ]:
%%time
scibert_model = BertTokenClassifier('scibert-scivocab-cased',
                            max_seq_length=185,
                            epochs=3,
                            gradient_accumulation_steps=2,
                            learning_rate=3e-5,
                            train_batch_size=16,
                            eval_batch_size=16,
                            validation_fraction=0.,                            
                            label_list=label_list,
                            ignore_label=['O'])



scibert_model.fit(X_train, y_train)

Building sklearn token classifier...


100%|██████████| 410521600/410521600 [00:13<00:00, 30220843.07B/s]


Loading scibert-scivocab-cased model...


100%|██████████| 410521600/410521600 [00:13<00:00, 30442497.16B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 981, validation data size: 0



...

Training  :   0%|          | 0/123 [00:00<?, ?it/s]

/content/bert-sklearn/bert_sklearn/model/pytorch_pretrained/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1055.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Training  :   0%|          | 0/123 [00:00<?, ?it/s]

Training  :   0%|          | 0/123 [00:00<?, ?it/s]

CPU times: user 2min 1s, sys: 46.5 s, total: 2min 47s
Wall time: 3min 41s


In [ ]:
scibert_f1_test = scibert_model.score(X_test, y_test,'macro')
print("Test f1: %0.02f"%(scibert_f1_test))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/32 [00:00<?, ?it/s]

Test f1: 76.60


In [31]:
scibert_y_preds = scibert_model.predict(X_test)
print(classification_report(flatten(y_test), flatten(scibert_y_preds)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/32 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  B-Chemical       0.95      0.75      0.84      5240
   B-Disease       0.85      0.71      0.77      4337
  I-Chemical       0.78      0.69      0.74       530
   I-Disease       0.75      0.69      0.72      2142
           O       0.97      0.99      0.98     95618

    accuracy                           0.96    107867
   macro avg       0.86      0.77      0.81    107867
weighted avg       0.96      0.96      0.96    107867



In [32]:
iter_zip = zip(flatten(X_test),flatten(y_test),flatten(scibert_y_preds))
preds = [" ".join([token, y, y_pred]) for token, y, y_pred in iter_zip]
with open("preds.txt",'w') as f:
    for x in preds:
        f.write(str(x)+'\n') 

!perl ./other_examples/conlleval.pl < preds.txt
!rm preds.txt

processed 107867 tokens with 9582 phrases; found: 7909 phrases; correct: 6769.
accuracy:  95.86%; precision:  85.59%; recall:  70.64%; FB1:  77.40
         Chemical: precision:  92.67%; recall:  73.83%; FB1:  82.19  4177
          Disease: precision:  77.65%; recall:  66.79%; FB1:  71.81  3732


In [ ]:
num = 28
tokens = X_test[num]
labels = y_test[num]
scibert_preds  = scibert_y_preds[num]

In [ ]:
data = {"token": tokens,"label": labels, "scibert": scibert_preds}
df=pd.DataFrame(data=data)
df.head(50)

,token,label,scibert
0,Syndrome,B-Disease,B-Disease
1,of,I-Disease,I-Disease
2,inappropriate,I-Disease,I-Disease
3,antidiuretic,I-Disease,I-Disease
4,hormone,I-Disease,I-Disease
5,secretion,O,I-Disease
6,associated,O,O
7,with,O,O
8,desvenlafaxine.,B-Chemical,B-Chemical
9,OBJECTIVE,O,O


### Hyperparameter Tuning

In [ ]:
# from sklearn.model_selection import GridSearchCV

# params = {'epochs':[3, 4], 'learning_rate':[2e-5, 3e-5, 5e-5]}

# scibert_clf = GridSearchCV(BertTokenClassifier('scibert-scivocab-cased',
#                             max_seq_length=256,
#                             gradient_accumulation_steps=2,
#                             train_batch_size=16,
#                             eval_batch_size=16,
#                             validation_fraction=0.,                            
#                             label_list=label_list,
#                             ignore_label=['O']), 
#                    params,
#                    verbose=True)

# scibert_clf.fit(X_train ,y_train)

# Results of DNER task with SciBERT

In [34]:
eval_data = pd.Series(eval_article.split('\n'))
tokenized_eval_data = eval_data.apply(lambda x:list(twt().tokenize(x)))
tokenized_eval_data

0      [Transductional, analysis, was, applied, to, the, Pseudomonas, aeruginosa, mutant, PAO14, (, hnc-1, ), ., This, mutant, can, utilize, L-histidinol, as, sole, source, of, carbon, and, nitrogen, and...
1      [Bacteriocins, produced, by, six, strains, of, Rhizobium, trifolii, were, found, to, be, of, the, relatively, low, molecular, weight, ,, non-phage, type., The, molecular, weights, ranged, from, ap...
2      [The, gamma, haemolysin, of, Staphylococcus, aureus, \'Smith, 5R\, ', was, produced, on, Dolman-Wilson, agar, overlain, with, cellophane., Maximal, yields, of, crude, lysin, with, titres, of, 2000...
3      [Arginine, and, methionine, transport, by, Aspergillus, nidulans, mycelium, was, investigated., A, single, uptake, system, is, responsible, for, the, transport, of, arginine, ,, lysine, and, ornit...
4      [Aphids, transmitted, poly-L-ornithine, (, PLO, ), -treated, tobacco, mosaic, virus, (, TMV, ), when, given, acquistion, and, inoculation, access, periods, as, brief

In [70]:
# # save the model to disk
scibert_model_file = '/content/scibert_model.pkl' #.sav
pickle.dump(scibert_model, open(scibert_model_file, 'wb'))

# load the model from disk
# loaded_model = pickle.load(open(scibert_model_file, 'r'))
# loaded_model = pd.read_pickle(open(scibert_model_file, 'r'))

In [38]:
# result = loaded_model.score(X_test, y_test)

In [ ]:
# files.download('/content/scibert_model.pkl')

In [37]:
eval_predicts_scibert = scibert_model.predict(tokenized_eval_data)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Predicting:   0%|          | 0/7 [00:00<?, ?it/s]

In [39]:
print(eval_predicts_scibert[4])

['O', 'O', 'B-Chemical', 'O', 'B-Chemical', 'O', 'O', 'B-Disease', 'I-Disease', 'I-Disease', 'O', 'B-Disease', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'I-Chemical', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'B-Disease', 'O', 'O', 'O', 'B-Disease', 'I-Disease', 'O', 'B-Chemical', 'B-Disease', 'O', 'O', 'O', 'O'

In [40]:
print(len(eval_predicts_scibert), len(eval_data.tolist()))

101 101


In [41]:
def locate_entities(text, annots):
  loc_list = []
  span_list = list(twt().span_tokenize(text))
  for annotation, loc in zip(annots, span_list):
    if annotation != 'O':
      loc = (loc[0], loc[1], str(annotation))
      loc_list.append(loc)
  return loc_list

In [42]:
eval_data_entities = [locate_entities(x,y) for x, y in zip(eval_data.tolist(), eval_predicts_scibert)]
eval_data_entities[99]

[(55, 61, 'B-Chemical'),
 (151, 156, 'B-Chemical'),
 (167, 178, 'B-Chemical'),
 (180, 189, 'B-Chemical'),
 (191, 198, 'B-Chemical'),
 (200, 231, 'B-Chemical'),
 (233, 246, 'B-Chemical'),
 (248, 254, 'B-Chemical'),
 (255, 272, 'I-Chemical'),
 (277, 283, 'B-Chemical'),
 (284, 307, 'B-Chemical'),
 (308, 312, 'I-Chemical'),
 (467, 478, 'B-Chemical'),
 (483, 496, 'B-Chemical'),
 (554, 560, 'B-Chemical'),
 (604, 625, 'B-Chemical'),
 (675, 697, 'B-Chemical'),
 (699, 706, 'B-Chemical'),
 (708, 717, 'B-Chemical'),
 (722, 727, 'B-Chemical'),
 (769, 774, 'B-Chemical'),
 (912, 919, 'B-Chemical'),
 (1072, 1087, 'B-Chemical')]

In [43]:
def display_entities(eval_data, num):
  if len(eval_data_entities[num]) == 0:
    return eval_data
  scibert_annot_data = [(eval_data.tolist()[num],
  {'entities': eval_data_entities[num]})]
  options = {"ents": ['B-Chemical', 'B-Disease','I-Chemical', 'I-Disease'], "colors": {'B-Chemical': "#85C1E9", "B-Disease": "#ff6961", 'I-Chemical': "#85C1E9", "I-Disease": "#ff6961"}}
  nlp = spacy.blank('en')
  raw_text = scibert_annot_data[0][0]
  doc = nlp.make_doc(raw_text)
  spans = scibert_annot_data[0][1]["entities"]
  ents = []
  for span_start, span_end, label in spans:
    ent = doc.char_span(span_start, span_end, label=label)
    if ent is None:
        continue
    ents.append(ent)
  doc.ents = ents
  displacy.render(doc, style="ent",options=options, jupyter=True)

In [74]:
display_entities(eval_data, 4)

In [ ]:
for i in range(5,15):
  display_entities(eval_data, i)

# Relation Extraction

### CID dataset

In [47]:
CID_train_df["text"] = CID_train_df.apply(lambda x: x['title_source_text']+ ' '+x['source_text'], axis = 1)
CID_test_df["text"] = CID_test_df.apply(lambda x: x['title_source_text']+ ' '+x['source_text'], axis = 1)
CID_dev_df["text"] = CID_dev_df.apply(lambda x: x['title_source_text']+ ' '+x['source_text'], axis = 1)

CID_train_df = CID_train_df.drop(columns = ['entity', 'name_id', 'title_source_text', 'source_text', 'xloc'])
CID_test_df = CID_test_df.drop(columns = ['entity', 'name_id', 'title_source_text', 'source_text', 'xloc'])
CID_dev_df = CID_dev_df.drop(columns = ['entity', 'name_id', 'title_source_text', 'source_text', 'xloc'])

In [48]:
CID_train_df = CID_train_df[CID_train_df['name'].str.startswith('D') & CID_train_df['yloc'].str.startswith('D')]
CID_test_df = CID_test_df[CID_test_df['name'].str.startswith('D') & CID_test_df['yloc'].str.startswith('D')]
CID_dev_df = CID_dev_df[CID_dev_df['name'].str.startswith('D') & CID_dev_df['yloc'].str.startswith('D')]

In [49]:
CID_train_df.head()

,text_id,yloc,name,text
0,354896,D008012,D006323,Lidocaine-induced cardiac asystole. Intravenous administration of a single 50-mg bolus of lidocaine in a 67-year-old man resulted in profound depression of the activity of the sinoatrial and atrio...
1,435349,D013390,D005207,Suxamethonium infusion rate and observed fasciculations. A dose-response study. Suxamethonium chloride (Sch) was administered i.v. to 36 adult males at six rates: 0.25 mg s-1 to 20 mg s-1. The inf...
2,603022,D012601,D062787,"Galanthamine hydrobromide, a longer acting anticholinesterase drug, in the treatment of the central effects of scopolamine (Hyoscine). Galanthamine hydrobromide, an anticholinesterase drug capable..."
3,1378968,D008094,D006973,"Effects of uninephrectomy and high protein feeding on lithium-induced chronic renal failure in rats. Rats with lithium-induced nephropathy were subjected to high protein (HP) feeding, uninephrecto..."
4,1378968,D008094,D011507,"Effects of uninephrectomy and high protein feeding on lithium-induced chronic renal failure in rats. Rats with lithium-induced nephropathy were subjected to high protein (HP) feeding, uninephrecto..."


In [50]:
DNER_train_df = DNER_train_df[DNER_train_df['name_id'].str.startswith('D') == True]
DNER_test_df = DNER_test_df[DNER_test_df['name_id'].str.startswith('D') == True]
DNER_dev_df = DNER_dev_df[DNER_dev_df['name_id'].str.startswith('D') == True]

In [51]:
test_names_df = pd.DataFrame(zip(DNER_train_df["name_id"].unique(), DNER_train_df['name'].unique()), columns=["name_id", "name"])

In [52]:
test_names_df.head()

,name_id,name
0,D009270,Naloxone
1,D003000,clonidine
2,D006973,hypertensive
3,D007022,hypotensive
4,D008750,alpha-methyldopa


In [53]:
DNER_train_df = DNER_train_df[DNER_train_df['xloc'].str.isnumeric() & DNER_train_df['yloc'].str.isnumeric()]
DNER_test_df = DNER_test_df[DNER_test_df['xloc'].str.isnumeric() & DNER_test_df['yloc'].str.isnumeric()]
DNER_dev_df = DNER_dev_df[DNER_dev_df['xloc'].str.isnumeric() & DNER_dev_df['yloc'].str.isnumeric()]

DNER_train_df = DNER_train_df.astype({"xloc": int, "yloc": int}, errors='raise')
DNER_test_df = DNER_test_df.astype({"xloc": int, "yloc": int}, errors='raise')
DNER_dev_df = DNER_dev_df.astype({"xloc": int, "yloc": int}, errors='raise')

DNER_train_df["text"] = DNER_train_df.apply(lambda x: x['title_source_text']+ ' '+x['source_text'], axis = 1)
DNER_train_df["loc"] = DNER_train_df.apply(lambda x: tuple((x['xloc'],x['yloc'])), axis = 1)

DNER_test_df['text'] = DNER_test_df.apply(lambda x: x['title_source_text']+ ' '+x['source_text'], axis = 1)
DNER_test_df["loc"] = DNER_test_df.apply(lambda x: tuple((x['xloc'],x['yloc'])), axis = 1)

DNER_dev_df['text'] = DNER_dev_df.apply(lambda x: x['title_source_text']+ ' '+x['source_text'], axis = 1)
DNER_dev_df["loc"] = DNER_dev_df.apply(lambda x: tuple((x['xloc'],x['yloc'])), axis = 1)

In [54]:
train_data_text_unp = pd.DataFrame(DNER_train_df["text"].unique(), columns=["sent"])
train_data_text_unp['tokens'] = train_data_text_unp.sent.apply(lambda x: list(twt().tokenize(x)))

test_data_text_unp = pd.DataFrame(DNER_test_df["text"].unique(), columns=["sent"])
test_data_text_unp['tokens'] = test_data_text_unp.sent.apply(lambda x: list(twt().tokenize(x)))

dev_data_text_unp = pd.DataFrame(DNER_dev_df["text"].unique(), columns=["sent"])
dev_data_text_unp['tokens'] = dev_data_text_unp.sent.apply(lambda x: list(twt().tokenize(x)))

In [55]:
def get_items( source_df, item="entity"):
  items_by_text = []
  for i in source_df["text"].unique():
    items_list = source_df[source_df["text"] == i][item].tolist()
    items_by_text.append(items_list)
  return items_by_text

def extend_df(df, source_d = DNER_train_df):
  df['entities'] = get_items(source_d,item="entity")
  df['names'] = get_items(source_d,item="name")
  df['loc'] = get_items(source_d,item="loc")
  df['name_ids'] = get_items(source_d,item="name_id")
  return df

In [56]:
train_data_text_unp = extend_df(train_data_text_unp, source_d = DNER_train_df)
test_data_text_unp = extend_df(test_data_text_unp, source_d = DNER_test_df)
dev_data_text_unp = extend_df(dev_data_text_unp, source_d = DNER_dev_df)

In [57]:
unique_train_data = train_data_text_unp['sent']

In [58]:
doc = nlp(unique_train_data.iloc[0])
for tok in doc:
  print(tok.text, "...", tok.dep_)
  break

Naloxone ... nsubj


### Knowledge Graphs

In [75]:
display_entities(eval_data, 4)

In [88]:
# tokenized_eval_data.iloc[4]

In [68]:
def get_entities(sent):
  entities = scibert_model.predict(sent)
  # This function should extract entity o=pairs from paragraph

In [89]:
# get_entities([tokenized_eval_data.iloc[4]])

In [45]:
# def get_entities(sent):
#   ent1 = ""
#   ent2 = ""

#   prv_tok_dep = ""    # dependency tag of previous token in the sentence
#   prv_tok_text = ""   # previous token in the sentence

#   prefix = ""
#   modifier = ""

#   for tok in nlp(sent):
#     # if token is a punctuation mark then move on to the next token
#     if tok.dep_ != "punct":
#       # check: token is a compound word or not
#       if tok.dep_ == "compound":
#         prefix = tok.text
#         # if the previous word was also a 'compound' then add the current word to it
#         if prv_tok_dep == "compound":
#           prefix = prv_tok_text + " "+ tok.text
      
#       # check: token is a modifier or not
#       if tok.dep_.endswith("mod") == True:
#         modifier = tok.text
#         # if the previous word was also a 'compound' then add the current word to it
#         if prv_tok_dep == "compound":
#           modifier = prv_tok_text + " "+ tok.text
      
#       if tok.dep_.find("subj") == True:
#         ent1 = modifier +" "+ prefix + " "+ tok.text
#         prefix = ""
#         modifier = ""
#         prv_tok_dep = ""
#         prv_tok_text = ""      

#       if tok.dep_.find("obj") == True:
#         ent2 = modifier +" "+ prefix +" "+ tok.text
        
#       prv_tok_dep = tok.dep_
#       prv_tok_text = tok.text

#   return [ent1.strip(), ent2.strip()]

In [90]:
# print(get_entities(unique_train_data.iloc[0]))
# print(get_entities(unique_train_data.iloc[5]))
# print(get_entities(unique_train_data.iloc[14]))

In [ ]:
# entity_pairs = []

# for i in tqdm(unique_train_data):
#   entity_pairs.append(get_entities(i))

100%|██████████| 489/489 [00:31<00:00, 15.31it/s]


In [ ]:
entity_pairs[46:57]

[['clinicians', 'unknown  origin'],
 ['widespread  triad', 'beta lactam antibiotics'],
 ['carefully side effects', 'particular  patient'],
 ['antithrombosis', 'mainly antiplatelet aggregation activities'],
 ['New chemotherapy combinations', 'advanced  NSCLC'],
 ['apparently calcification they', 'calcification sites'],
 ['hyperthymic  who', 'greater mood switches'],
 ['high guarana health case', 'such  products'],
 ['2 respiratory cases', 'continuous dilatation mimicking fentanyl'],
 ['cumulative  HAART', 'LA transgenic mice'],
 ['oral  they', 'prothrombotic  mutation']]

In [60]:
sent_tokenized_eval_data = eval_data.apply(lambda x:x.split('. '))
sent_tokenized_eval_data.iloc[2]

["The gamma haemolysin of Staphylococcus aureus \\'Smith 5R\\' was produced on Dolman-Wilson agar overlain with cellophane",
 'Maximal yields of crude lysin with titres of 2000 to 4000 haemolytic units/ml were obtained within 24 h at 37 degrees C in 10% (v/v) CO2 in air, on medium adjusted to pH 7-0',
 'The crude lysin was purified 2700-fold (with 75% recovery) by ultrafiltration, gel filtration and ammonium sulphate fractionation',
 'The specific activity of the lysin was 10(5) haemolytic units/mg protein after the dialysed active precipitate was extracted with NaCl and reprecipitated with ammonium sulphate',
 'Purified gamma lysin was homogeneous by disc electrophoresis and immunoelectrophoresis.']

In [ ]:
sent_tokenized_eval_data = pd.DataFrame(sent_tokenized_eval_data, columns= ["paragraph"])

In [ ]:
sent_tokenized_eval_data['entity pairs'] = sent_tokenized_eval_data["paragraph"].apply(lambda x: [get_entities(i) for i in x])

In [ ]:
sent_tokenized_eval_data

,paragraph,entity pairs
0,"[Transductional analysis was applied to the Pseudomonas aeruginosa mutant PAO14 (hnc-1), This mutant can utilize L-histidinol as sole source of carbon and nitrogen and has a 60-fold increased hist...","[[Transductional analysis, aeruginosa mutant PAO14], [mutant, HDH content], [where hnc-1, histidine locus biosynthesis], [at group IV which, structural HDH], [genetic hnc-1 mutation, structura..."
1,"[Bacteriocins produced by six strains of Rhizobium trifolii were found to be of the relatively low molecular weight, non-phage type, The molecular weights ranged from approximately 1-8 X 105 to 2-...","[[Bacteriocins, phage Rhizobium type], [molecular weights, 1 X], [All, proteolytic protein enzymes], [They, DNAase], [six bacteriocins, sensitive activity R], [, sensitive bacteria], [Bacterio..."
2,"[The gamma haemolysin of Staphylococcus aureus \'Smith 5R\' was produced on Dolman-Wilson agar overlain with cellophane, Maximal yields of crude lysin with titres of 2000 to 4000 haemolytic units/...","[[gamma haemolysin, Dolman Wilson cellophane], [haemolytic ml, medium degrees pH], [crude lysin, 75 % ultrafiltration], [active precipitate, ammonium sulphate], [Purified gamma lysin, disc elec..."
3,"[Arginine and methionine transport by Aspergillus nidulans mycelium was investigated, A single uptake system is responsible for the transport of arginine, lysine and ornithine, Transport is energy...","[[Arginine, Aspergillus nidulans mycelium], [single uptake system, arginine], [Transport, basic amino acids], [12 Ki, ornithine], [wt transport, azide], [Neutral amnio acids, specifically methio..."
4,"[Aphids transmitted poly-L-ornithine (PLO)-treated tobacco mosaic virus (TMV) when given acquistion and inoculation access periods as brief as 30 s and 2 min, respectively; the ability to transmit...","[[respectively access ability, 90 min], [Aphids, virus], [Transmission, nonpersistent viruses], [, well KCl concentration], [which, ml TMV], [, ], [, ], [similar mixture, transmission], [, ], ..."
...,...,...
96,"[Most-probable-number (MPN) and membrane filtration (mF) techniques were evaluated with respect to selectivity, sensitivity, and efficiency in recovering Pseudomonas aeruginosa strains in hospital...","[[mF membrane techniques, hospital fluids], [, occurring P], [aeruginosa, microbial Standard Methods contamiants], [, unknown P], [aeruginosa strains, ], [mPA MPN agars, mF assays], [Statistical..."
97,[The relationship between muscle surface pH (pHM) and the arterial-venous oxygen content difference (AVO2D) was studied in 4 patients undergoing reconstructive arterial surgery and in 6 patients u...,"[[relationship, normovolemic oxygen content hemodilution], [, aortic inverse clamping], [, also inverse hemodilution], [pHM, reliable tissue AVO2D.]]"
98,"[The specificity of Ca++ for the interaction of beta adrenergic agonists with their receptors in rabbit right atrial muscle was evaluated, This was accomplished by substituting Ca++ by an equimola...","[[atrial muscle, adrenergic rabbit], [This, equimolar Sr++], [which, electrical Ca++], [10(-7 Sr++-containing medium, 10(-7 Ca++-containing], [clearly Sr++, cardiac muscle], [Also norepine..."
99,"[The action of a number of compounds able to react with thiols was tested on guinea-pig hearts perfused at constant pressure, The SH reagents used were NaNO2, oxidized glutathione, cystamine, diam...","[[action, constant guinea pig pressure], [glutathione, ], [SH that, effect], [other SH reagents, coronary flow], [oxidized, heart rate], [completely oxygen increase, coronary flow], [Difluorodin..."


In [ ]:
def get_relation(sent):
  matcher = Matcher(nlp.vocab)
  patterns = [[{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}]]

  matcher.add("matching_1", patterns) 
  doc = nlp(sent)
  matches = matcher(doc)
  if len(matches)>0:
    k = len(matches) - 1
    span = doc[matches[k][1]:matches[k][2]] 
  else: return ''
  return(span.text)

In [ ]:
get_relation("John completed the task")

'completed'

In [ ]:
sent_tokenized_eval_data['relations'] = sent_tokenized_eval_data["paragraph"].apply(lambda x: [get_relation(i) for i in x])

In [ ]:
sent_tokenized_eval_data.head(50)

,paragraph,entity pairs,relations
0,"[Transductional analysis was applied to the Pseudomonas aeruginosa mutant PAO14 (hnc-1), This mutant can utilize L-histidinol as sole source of carbon and nitrogen and has a 60-fold increased hist...","[[Transductional analysis, aeruginosa mutant PAO14], [mutant, HDH content], [where hnc-1, histidine locus biosynthesis], [at group IV which, structural HDH], [genetic hnc-1 mutation, structura...","[applied to, utilize, carried, marker, are in full, suggested]"
1,"[Bacteriocins produced by six strains of Rhizobium trifolii were found to be of the relatively low molecular weight, non-phage type, The molecular weights ranged from approximately 1-8 X 105 to 2-...","[[Bacteriocins, phage Rhizobium type], [molecular weights, 1 X], [All, proteolytic protein enzymes], [They, DNAase], [six bacteriocins, sensitive activity R], [, sensitive bacteria], [Bacterio...","[found, ranged from, were of, were resistant, separated into, trifolii, occurred]"
2,"[The gamma haemolysin of Staphylococcus aureus \'Smith 5R\' was produced on Dolman-Wilson agar overlain with cellophane, Maximal yields of crude lysin with titres of 2000 to 4000 haemolytic units/...","[[gamma haemolysin, Dolman Wilson cellophane], [haemolytic ml, medium degrees pH], [crude lysin, 75 % ultrafiltration], [active precipitate, ammonium sulphate], [Purified gamma lysin, disc elec...","[produced on, obtained within, purified, was, was homogeneous]"
3,"[Arginine and methionine transport by Aspergillus nidulans mycelium was investigated, A single uptake system is responsible for the transport of arginine, lysine and ornithine, Transport is energy...","[[Arginine, Aspergillus nidulans mycelium], [single uptake system, arginine], [Transport, basic amino acids], [12 Ki, ornithine], [wt transport, azide], [Neutral amnio acids, specifically methio...","[investigated, is responsible, is, is, inhibited by, transported by, isolated as resistant, has unchanged]"
4,"[Aphids transmitted poly-L-ornithine (PLO)-treated tobacco mosaic virus (TMV) when given acquistion and inoculation access periods as brief as 30 s and 2 min, respectively; the ability to transmit...","[[respectively access ability, 90 min], [Aphids, virus], [Transmission, nonpersistent viruses], [, well KCl concentration], [which, ml TMV], [, ], [, ], [similar mixture, transmission], [, ], ...","[lost within, were able, seems similar, ratio of, was best, wt, 120000, favoured, wt, substituted for, wt, was necessary, favoured, remained aphid, show, suggest, ruled]"
5,"[It has been shown that particles of Vi bacteriophage III catalyse deacetylation of O-acetyl pectic (polygalacturonic) acid, a structural analogue of Vi polysaccharide (Vi antigen), Using this sub...","[[It, structural Vi polysaccharide], [liquid method, phage deacetylase activity], [Purified particles, plaque deacetylase activity], [They, also electron microscope], [Osmotic shock, still base...","[shown that, developed, exposed to, inspected under, found, exhibited, fractionated, showed, with, X, found in]"
6,"[The authors measured regional cerebral 133xenon (133Xe) blood flow (rCBF), intraventricular pressure (IVP), cerebrospinal fluid (CSF) pH and lactate, systemic arterial blood pressure (SAP), and a...","[[authors, severe head injuries], [IVP, ], [repeatedly response, induced hypertension], [Most patients, ], [correlation, reduced brain function], [correlation, CBF], [CSF lactate, cortical brai...","[measured regional, Hg, measured, reduced, found between average, found between, elevated, had, referred to, found in, increased in, provoked by induced, appeared]"
7,[Deans of Nursing who choose to be administrators of the Health Sciences may very well enhance Nursing programs by facilitating the nursing faculty to become leaders through offering service cours...,"[[who, special Health theory]]",[enhance]
8,"[A basal diet or a basal diet plus 1% of cholesterol and 0.33% cholic acid was fed to rats for varying lengths of time and (

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
relations = flatten(sent_tokenized_eval_data['relations'].tolist())
entity_pairs = flatten(sent_tokenized_eval_data['entity pairs'].tolist())

In [93]:
# source = [i[0] for i in entity_pairs]

# target = [i[1] for i in entity_pairs]

# kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})

In [ ]:
# G=nx.from_pandas_edgelist(kg_df, "source", "target", 
#                           edge_attr=True, create_using=nx.MultiDiGraph())

In [94]:
# plt.figure(figsize=(12,12))

# pos = nx.spring_layout(G)
# nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
# plt.show()

In [ ]:
# [i for i in relations if i!='']

In [92]:
# G=nx.from_pandas_edgelist(kg_df[kg_df['edge']=='is'], "source", "target", 
#                           edge_attr=True, create_using=nx.MultiDiGraph())

# plt.figure(figsize=(12,12))
# pos = nx.spring_layout(G, k = 0.7) # k regulates the distance between nodes
# nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
# plt.show()

In [91]:
# G=nx.from_pandas_edgelist(kg_df[kg_df['edge']=='applied to'], "source", "target", 
#                           edge_attr=True, create_using=nx.MultiDiGraph())

# plt.figure(figsize=(6, 6))
# pos = nx.spring_layout(G, k = 0.7)
# nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
# plt.show()